In [165]:
import numpy as np
import os

In [166]:
%pwd

'C:\\Users\\llaur\\OneDrive\\Desktop\\turb\\proj-turbulence\\test'

In [167]:
%cd ../../../../../Downloads/ERA_wind

C:\Users\llaur\Downloads\ERA_wind


In [168]:
fine = np.load("ERA_wind.npz")

# Load

## Train

In [169]:
%cd ../../OneDrive/Desktop/turb/proj-turbulence/train

C:\Users\llaur\OneDrive\Desktop\turb\proj-turbulence\train


In [170]:
u = fine.get('wind_u')
v = fine.get('wind_v')

In [171]:
# for lat in range(121)
#     for lon in range(201):
#         # the set of wind speeds at all time stamps at this one point
#         wind_speeds = [np.sqrt((u[i][1][lat][lon])**2 + (v[i][1][lat][lon])**2) for i in range (744)]

List comprehension of wind speeds at level 225

In [172]:
wind_speeds = [[[np.sqrt((u[i][1][lat][lon])**2 + (v[i][1][lat][lon])**2) for lon in range(200)] for lat in range(120)] for i in range(744)]

In [174]:
len(wind_speeds), len(wind_speeds[0]), len(wind_speeds[0][0])

(744, 120, 200)

In [182]:
files = os.listdir(os.getcwd())

Add to coarse and fine lists in order

In [184]:
coarse_train = []
fine_train = []
for file in files: # loop thru files
    if file[0]=="w":
        timestamp = file[19:-4]
        coarse_train.append(np.load(file)) # load file, save into coarse
        fine_train.append(wind_speeds[int(timestamp)]) # load fine version

In [185]:
import json
with open("wind_speeds.json", "w") as file:
    json.dump(wind_speeds, file)

KeyboardInterrupt: 

595 samples, # lat points, # lon points

In [186]:
len(coarse_train), len(coarse_train[0]), len(coarse_train[0][0])

(595, 15, 25)

In [187]:
len(fine_train), len(fine_train[0]), len(fine_train[0][0])

(595, 120, 200)

## Val

In [188]:
%cd ../val

C:\Users\llaur\OneDrive\Desktop\turb\proj-turbulence\val


In [189]:
files = os.listdir(os.getcwd())
coarse_val = []
fine_val = []
for file in files: # loop thru files
    if file[0]=="w":
        timestamp = file[19:-4]
        coarse_val.append(np.load(file)) # load file, save into coarse
        fine_val.append(wind_speeds[int(timestamp)]) # load fine version

In [190]:
len(coarse_val), len(coarse_val[0]), len(coarse_val[0][0])

(75, 15, 25)

In [191]:
len(fine_val), len(fine_val[0]), len(fine_val[0][0])

(75, 120, 200)

## Test

In [192]:
%cd ../test

C:\Users\llaur\OneDrive\Desktop\turb\proj-turbulence\test


In [193]:
files = os.listdir(os.getcwd())
coarse_test = []
fine_test = []
for file in files: # loop thru files
    if file[0]=="w":
        timestamp = file[19:-4]
        coarse_test.append(np.load(file)) # load file, save into coarse
        fine_test.append(wind_speeds[int(timestamp)]) # load fine version

In [194]:
len(coarse_test), len(coarse_test[0]), len(coarse_test[0][0])

(74, 15, 25)

In [195]:
len(fine_test), len(fine_test[0]), len(fine_test[0][0]), type(fine_test[0][0][0])

(74, 120, 200, numpy.float64)

# CNN

In [196]:
import torch.nn as nn
from torchvision import transforms
from torch.utils.data import DataLoader, Dataset
import torch.optim as optim

## Load Data

In [241]:
class PairedImageDataset(Dataset):
    def __init__(self, coarse_images, high_res_images, transform=None):
        self.coarse_images = coarse_images
        self.high_res_images = high_res_images
        self.transform = transform

    def __len__(self):
        return len(self.coarse_images)

    def __getitem__(self, idx):
        coarse_img = self.coarse_images[idx]
        high_res_img = self.high_res_images[idx]

        if self.transform:
            coarse_img = self.transform(coarse_img)
            high_res_img = self.transform(high_res_img)

        return coarse_img, high_res_img

transform = transforms.Compose([
    transforms.ToTensor(),  # Convert images to PyTorch tensors
])

train = PairedImageDataset(np.array(coarse_train), np.array(fine_train), transform=transform)
test = PairedImageDataset(np.array(coarse_test), np.array(fine_test), transform=transform)
val = PairedImageDataset(np.array(coarse_val), np.array(fine_val), transform=transform)

# load
batch_size = 16
train_loader = DataLoader(train, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val, batch_size=batch_size, shuffle=True)

In [242]:
np.array(coarse_train).shape, np.array(fine_train).shape

((595, 15, 25), (595, 120, 200))

## Define CNN

In [243]:
class CNN(nn.Module):
    def __init__(self):
        upscale = 8
        super(CNN, self).__init__()
        self.conv_layers = nn.Sequential(
            nn.Conv2d(1, 16, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv2d(16, 16, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv2d(16, 16, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv2d(16, 16, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv2d(16, upscale**2, kernel_size=3, padding=1),
            nn.ReLU()
        )
        self.depth_to_space = nn.PixelShuffle(upscale)
        
    def forward(self, x):
        x = self.conv_layers(x.float())
        x = self.depth_to_space(x)
        return x
    
model = CNN()

## Train CNN

In [249]:
import torch

In [250]:
criterion = nn.MSELoss()
optimizer = optim.NAdam(cnn.parameters(), lr=0.00000000000000000000000001)

# Training loop
num_epochs = 500
for epoch in range(num_epochs):
    model.train()
    for inputs, targets in train_loader:
        optimizer.zero_grad()
        
#         inputs = inputs.squeeze()
        targets = targets.squeeze() 
#         print("shapes::::", inputs.shape, targets.shape)
        outputs = model(inputs)
        
        loss = criterion(outputs, targets.squeeze().float())
        loss.backward()
        optimizer.step()
    # Validation
    model.eval()
    with torch.no_grad():
        for inputs, targets in val_loader:
            outputs = model(inputs)
            val_loss = criterion(outputs, targets.squeeze())
    print(f'Epoch {epoch+1}/{num_epochs}, Loss: {loss.item()}, Val Loss: {val_loss.item()}')


Epoch 1/500, Loss: 558.052734375, Val Loss: 541.7008411092088
Epoch 2/500, Loss: 627.2586059570312, Val Loss: 512.0131937266107
Epoch 3/500, Loss: 421.6134033203125, Val Loss: 493.8916035939348
Epoch 4/500, Loss: 439.005615234375, Val Loss: 494.602965867935
Epoch 5/500, Loss: 432.7286682128906, Val Loss: 539.804745825562
Epoch 6/500, Loss: 541.2561645507812, Val Loss: 540.9807608916528
Epoch 7/500, Loss: 503.9337463378906, Val Loss: 528.66395821901
Epoch 8/500, Loss: 637.3790893554688, Val Loss: 518.8255775813224
Epoch 9/500, Loss: 553.5615844726562, Val Loss: 591.1838148970093
Epoch 10/500, Loss: 513.9009399414062, Val Loss: 544.73562691653
Epoch 11/500, Loss: 751.6361694335938, Val Loss: 464.0344192929624
Epoch 12/500, Loss: 556.069091796875, Val Loss: 564.7656573373757
Epoch 13/500, Loss: 675.9747314453125, Val Loss: 524.1569010990088
Epoch 14/500, Loss: 399.5264892578125, Val Loss: 570.6912068938872
Epoch 15/500, Loss: 620.0687866210938, Val Loss: 520.7096576836695
Epoch 16/500, Lo

Epoch 124/500, Loss: 633.794677734375, Val Loss: 593.3012163569251
Epoch 125/500, Loss: 493.19232177734375, Val Loss: 536.2586574454835
Epoch 126/500, Loss: 567.9945068359375, Val Loss: 478.9677832604891
Epoch 127/500, Loss: 472.8819274902344, Val Loss: 564.6824434267171
Epoch 128/500, Loss: 594.5746459960938, Val Loss: 525.9294417700177
Epoch 129/500, Loss: 557.1177978515625, Val Loss: 534.1133834911135
Epoch 130/500, Loss: 481.644287109375, Val Loss: 565.1030779405966
Epoch 131/500, Loss: 646.2188720703125, Val Loss: 534.0332220561153
Epoch 132/500, Loss: 461.5723571777344, Val Loss: 582.681277115425
Epoch 133/500, Loss: 605.5538330078125, Val Loss: 509.3901981928063
Epoch 134/500, Loss: 632.7379150390625, Val Loss: 557.5695569700764
Epoch 135/500, Loss: 581.7783813476562, Val Loss: 487.58639552027546
Epoch 136/500, Loss: 605.5850830078125, Val Loss: 504.8478000519384
Epoch 137/500, Loss: 582.7991333007812, Val Loss: 519.5891466939571
Epoch 138/500, Loss: 454.60235595703125, Val Loss

Epoch 246/500, Loss: 540.4950561523438, Val Loss: 498.58976988310894
Epoch 247/500, Loss: 495.836669921875, Val Loss: 525.6029539728906
Epoch 248/500, Loss: 539.7841796875, Val Loss: 522.618744097931
Epoch 249/500, Loss: 553.9036865234375, Val Loss: 450.1742231922154
Epoch 250/500, Loss: 622.9188232421875, Val Loss: 521.4479406281804
Epoch 251/500, Loss: 589.5850830078125, Val Loss: 542.8212893421439
Epoch 252/500, Loss: 557.254150390625, Val Loss: 523.378388186698
Epoch 253/500, Loss: 593.3197021484375, Val Loss: 568.6989319143788
Epoch 254/500, Loss: 512.8455200195312, Val Loss: 521.2565537199077
Epoch 255/500, Loss: 615.5069580078125, Val Loss: 564.1797124876693
Epoch 256/500, Loss: 677.3341674804688, Val Loss: 527.315999658588
Epoch 257/500, Loss: 438.55548095703125, Val Loss: 582.6657138736155
Epoch 258/500, Loss: 529.1700439453125, Val Loss: 519.0679714355666
Epoch 259/500, Loss: 594.4030151367188, Val Loss: 520.0934080864021
Epoch 260/500, Loss: 507.5894775390625, Val Loss: 589.

Epoch 367/500, Loss: 497.1742248535156, Val Loss: 569.8825847032133
Epoch 368/500, Loss: 416.5218200683594, Val Loss: 584.6950686790434
Epoch 369/500, Loss: 547.783935546875, Val Loss: 559.5193517187222
Epoch 370/500, Loss: 446.0861511230469, Val Loss: 504.7225798840112
Epoch 371/500, Loss: 511.6707458496094, Val Loss: 524.7864585085191
Epoch 372/500, Loss: 475.0406799316406, Val Loss: 439.6550724154088
Epoch 373/500, Loss: 461.11651611328125, Val Loss: 532.177598407701
Epoch 374/500, Loss: 546.159912109375, Val Loss: 535.2571436065394
Epoch 375/500, Loss: 555.4522094726562, Val Loss: 517.593743522084
Epoch 376/500, Loss: 460.5572509765625, Val Loss: 476.680900922825
Epoch 377/500, Loss: 519.371826171875, Val Loss: 545.7331525834535
Epoch 378/500, Loss: 624.8457641601562, Val Loss: 535.0205027699673
Epoch 379/500, Loss: 580.8557739257812, Val Loss: 524.201445231544
Epoch 380/500, Loss: 425.40008544921875, Val Loss: 465.29480759147793
Epoch 381/500, Loss: 588.3653564453125, Val Loss: 56

KeyboardInterrupt: 